<a href="https://colab.research.google.com/github/OmarAbdelaziz0/StudentGradesAndPrograms/blob/main/StudentGradesAndPrograms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: Import StudentGradesAndPrograms.csv file

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Assuming the file is in your Google Drive, replace with the correct path
file_path = "/content/drive/MyDrive/StudentGradesAndPrograms.csv"

try:
  df = pd.read_csv(file_path)
  print(df)
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
except pd.errors.ParserError:
  print(f"Error: Could not parse the CSV file at {file_path}. Check the file format.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")
  print(df.head())
  print(df.info())
  print(df.describe())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       schoolyear gradeLevel classPeriod classType  \
0       2024-2025         07           1       ELE   
1       2024-2025         07           1       ELE   
2       2024-2025         07           1       ELE   
3       2024-2025         07           1       ELE   
4       2024-2025         07           1       ELE   
...           ...        ...         ...       ...   
200989  2024-2025         06           1       ELE   
200990  2024-2025         07           3       ENG   
200991  2024-2025         05           7       SOC   
200992  2024-2025         04           8       SCI   
200993  2024-2025         07           2       SCI   

                           schoolName  gradePercentage avid sped migrant ell  \
0                    West Junior High           2000.0    Y    N       N   N   
1                    West Junior High           2000.0    N   

<ipython-input-2-516522e31da8>:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
print(df.head())
print(df.info()) #Dataset information
print(df.describe())

  schoolyear gradeLevel classPeriod classType        schoolName  \
0  2024-2025         07           1       ELE  West Junior High   
1  2024-2025         07           1       ELE  West Junior High   
2  2024-2025         07           1       ELE  West Junior High   
3  2024-2025         07           1       ELE  West Junior High   
4  2024-2025         07           1       ELE  West Junior High   

   gradePercentage avid sped migrant ell student_ID  
0           2000.0    Y    N       N   N  0HRJHI993  
1           2000.0    N    N       N   N  CKN322II4  
2           1950.0    N    N       N   N  V523OZUH8  
3           1850.0    Y    N       N   N  OJDYS3434  
4           1500.0    N    N       Y   Y  49RSM3UF6  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200994 entries, 0 to 200993
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   schoolyear       200994 non-null  object 
 1   gradeLevel      

In [ ]:
print("Missing values:\n", df.isnull().sum())


Missing values:
 schoolyear         0
gradeLevel         0
classPeriod        0
classType          0
schoolName         0
gradePercentage    0
avid               0
sped               0
migrant            0
ell                0
student_ID         0
dtype: int64


In [ ]:
df = df.dropna()
df = df.drop_duplicates()
df = pd.get_dummies(df, drop_first=True)  # Creates dummy variables for categorical columns (One Hot Encoding)




In [ ]:
# prompt: normalize numerical data for performance

from google.colab import drive
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

drive.mount('/content/drive')

# Assuming the file is in your Google Drive, replace with the correct path
file_path = "/content/drive/MyDrive/StudentGradesAndPrograms.csv"

try:
  df = pd.read_csv(file_path)
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
  exit() #Exit the script if the file is not found
except pd.errors.ParserError:
  print(f"Error: Could not parse the CSV file at {file_path}. Check the file format.")
  exit() #Exit the script if there's a parsing error
except Exception as e:
  print(f"An unexpected error occurred: {e}")
  exit() #Exit the script if an unexpected error occurs


# Identify numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit and transform the numerical columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print(df.head())
print(df.info()) #Dataset information
print(df.describe())
print("Missing values:\n", df.isnull().sum())

df = df.dropna()
df = df.drop_duplicates()
df = pd.get_dummies(df, drop_first=True)  # Creates dummy variables for categorical columns (One Hot Encoding)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-6-b8b0753a39a1>:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


  schoolyear gradeLevel classPeriod classType        schoolName  \
0  2024-2025         07           1       ELE  West Junior High   
1  2024-2025         07           1       ELE  West Junior High   
2  2024-2025         07           1       ELE  West Junior High   
3  2024-2025         07           1       ELE  West Junior High   
4  2024-2025         07           1       ELE  West Junior High   

   gradePercentage avid sped migrant ell student_ID  
0            1.000    Y    N       N   N  0HRJHI993  
1            1.000    N    N       N   N  CKN322II4  
2            0.975    N    N       N   N  V523OZUH8  
3            0.925    Y    N       N   N  OJDYS3434  
4            0.750    N    N       Y   Y  49RSM3UF6  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200994 entries, 0 to 200993
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   schoolyear       200994 non-null  object 
 1   gradeLevel      

In [ ]:
# prompt: Split the dataset into features (x) and target (y)

# Assuming 'df' is your DataFrame after preprocessing
# Separate features (x) and target (y)
x = df.drop('gradePercentage', axis=1)  # Replace 'Grade' with the actual name of your target column
y = df['gradePercentage']

In [ ]:
# prompt: Split the dataset into training and testing sets

from sklearn.model_selection import train_test_split

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) # 80% training and 20% test

In [ ]:
# prompt: Start with a simple linear regression model:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = model.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")